In [ ]:
import sqlite3
import os
import hashlib
import streamlit as st
import requests

/Users/re.v/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# API Premium
API_KEY = "609380"
# NFL ID = 4391
LEAGUE_ID = 4391
# Path
DB = "/Users/re.v/Documents/quiniela/quiniela.db"

# Crear bases de datos: Usuarios, Partidos, Predicciones, Puntajes

In [ ]:
def create_database(db_path=DB):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    sql_statements = [
        """
        CREATE TABLE IF NOT EXISTS usuarios (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            password_hash TEXT NOT NULL,
            fecha_registro DATETIME DEFAULT CURRENT_TIMESTAMP
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS equipos (
            team_id TEXT PRIMARY KEY,
            nombre TEXT NOT NULL,
            badge_url TEXT,
            logo_url TEXT
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS partidos (
            partido_id INTEGER PRIMARY KEY AUTOINCREMENT,
            external_id TEXT UNIQUE,
            semana INTEGER NOT NULL,
            fecha DATETIME,
            equipo_local TEXT NOT NULL,
            equipo_visitante TEXT NOT NULL,
            estadio TEXT,
            status TEXT DEFAULT 'scheduled'
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS predicciones (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            usuario_id INTEGER NOT NULL,
            partido_id INTEGER NOT NULL,
            semana INTEGER NOT NULL,
            pick TEXT NOT NULL,
            fecha_prediccion DATETIME DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(usuario_id, partido_id),
            FOREIGN KEY(usuario_id) REFERENCES usuarios(id),
            FOREIGN KEY(partido_id) REFERENCES partidos(partido_id)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS resultados (
            partido_id INTEGER PRIMARY KEY,
            external_id TEXT UNIQUE,
            semana INTEGER NOT NULL,
            score_local INTEGER,
            score_visitante INTEGER,
            ganador TEXT,
            status TEXT DEFAULT 'pending',
            last_update DATETIME DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY(partido_id) REFERENCES partidos(partido_id)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS puntajes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            usuario_id INTEGER NOT NULL,
            semana INTEGER NOT NULL,
            puntos INTEGER NOT NULL DEFAULT 0,
            updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(usuario_id, semana),
            FOREIGN KEY(usuario_id) REFERENCES usuarios(id)
        );
        """,
        "CREATE INDEX IF NOT EXISTS idx_partidos_semana ON partidos(semana);",
        "CREATE INDEX IF NOT EXISTS idx_predicciones_usuario ON predicciones(usuario_id);",
        "CREATE INDEX IF NOT EXISTS idx_equipos_nombre ON equipos(nombre);",
        "CREATE INDEX IF NOT EXISTS idx_resultados_semana ON resultados(semana);",
        "CREATE INDEX IF NOT EXISTS idx_puntajes_usuario ON puntajes(usuario_id);"
    ]

    for s in sql_statements:
        cur.executescript(s)

    conn.commit()
    conn.close()
    print(f"Base creada en: {db_path}")


In [18]:
create_database()

Base creada en: /Users/re.v/Documents/quiniela/quiniela.db


### Insertar datos

In [3]:
import sqlite3

conn = sqlite3.connect("/Users/re.v/Documents/quiniela/quiniela.db")
cur = conn.cursor()

cur.execute("SELECT COUNT(*) FROM equipos")
print(cur.fetchone())

conn.close()


(0,)


In [ ]:
# Insertar usuarios

def hash_password(password: str) -> str:
    return hashlib.sha256(password.encode()).hexdigest()

def add_user(nombre, email, password):
    conn = sqlite3.connect(DB)
    cur = conn.cursor()
    try:
        cur.execute("""
            INSERT INTO usuarios (nombre, email, password_hash)
            VALUES (?, ?, ?)
        """, (nombre, email, hash_password(password)))
        
        conn.commit()
        print("Usuario agregado.")
    except sqlite3.IntegrityError:
        print("⚠️ Ese email ya está registrado.")
    finally:
        conn.close()

# Insertar partidos 

def add_partido(external_id, semana, fecha, local, visitante, estadio):
    conn = sqlite3.connect(DB)
    cur = conn.cursor()
    try:
        cur.execute("""
            INSERT INTO partidos (external_id, semana, fecha, equipo_local, equipo_visitante, estadio)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (external_id, semana, fecha, local, visitante, estadio))
        
        conn.commit()
        print("Partido agregado.")
    except sqlite3.IntegrityError:
        print("⚠️ Ese partido ya existe (external_id duplicado).")
    finally:
        conn.close()

# Agregar predicciones

def add_prediccion(usuario_id, partido_id, semana, pick):
    conn = sqlite3.connect(DB)
    cur = conn.cursor()
    try:
        cur.execute("""
            INSERT INTO predicciones (usuario_id, partido_id, semana, pick)
            VALUES (?, ?, ?, ?)
        """, (usuario_id, partido_id, semana, pick))
        
        conn.commit()
        print("Predicción guardada.")
    except sqlite3.IntegrityError:
        print("⚠️ Ya existe una predicción para ese usuario y partido.")
    finally:
        conn.close()

# Consultar partidos

def get_partidos(semana=None):
    conn = sqlite3.connect(DB)
    cur = conn.cursor()

    if semana:
        cur.execute("SELECT * FROM partidos WHERE semana = ?", (semana,))
    else:
        cur.execute("SELECT * FROM partidos")

    rows = cur.fetchall()
    conn.close()
    return rows

# Conexión del API

In [ ]:
url = "https://www.thesportsdb.com/api/v2/json/search/league/NFL"
headers = {"X-API-KEY": API_KEY}

r = requests.get(url, headers=headers)
print(r.status_code)
print(r.json())

200
{'search': [{'idLeague': '4391', 'strLeague': 'NFL', 'strSport': 'American Football', 'strBadge': 'https://r2.thesportsdb.com/images/media/league/badge/g85fqz1662057187.png', 'strCountry': 'United States', 'strCurrentSeason': '2025', 'strGender': 'Male'}]}


In [ ]:
url = f"https://www.thesportsdb.com/api/v2/json/schedule/next/league/{LEAGUE_ID}"
headers = {"X-API-KEY": API_KEY}

r = requests.get(url, headers=headers)
data = r.json()

games = data.get("schedule", [])

print(f"Próximos partidos NFL ({len(games)})\n")

for g in games:
    fecha = g.get("dateEvent")
    home = g.get("strHomeTeam")
    away = g.get("strAwayTeam")
    venue = g.get("strVenue")

    print(f"{fecha} | {away} @ {home} | {venue}")


Próximos partidos NFL (20)

2025-12-14 | Cleveland Browns @ Chicago Bears | Soldier Field
2025-12-14 | Baltimore Ravens @ Cincinnati Bengals | Paycor Stadium
2025-12-14 | Los Angeles Chargers @ Kansas City Chiefs | GEHA Field at Arrowhead Stadium
2025-12-14 | Buffalo Bills @ New England Patriots | Gillette Stadium
2025-12-14 | Washington Commanders @ New York Giants | MetLife Stadium
2025-12-14 | Las Vegas Raiders @ Philadelphia Eagles | Lincoln Financial Field
2025-12-14 | New York Jets @ Jacksonville Jaguars | EverBank Stadium
2025-12-14 | Arizona Cardinals @ Houston Texans | NRG Stadium
2025-12-14 | Green Bay Packers @ Denver Broncos | Empower Field at Mile High
2025-12-14 | Detroit Lions @ Los Angeles Rams | SoFi Stadium
2025-12-14 | Carolina Panthers @ New Orleans Saints | Caesars Superdome
2025-12-14 | Tennessee Titans @ San Francisco 49ers | Levi's Stadium
2025-12-14 | Indianapolis Colts @ Seattle Seahawks | Lumen Field
2025-12-15 | Minnesota Vikings @ Dallas Cowboys | AT&T Stad

## Conexión de API a SQL 

In [ ]:
def save_next_games():
    url = f"https://www.thesportsdb.com/api/v2/json/schedule/next/league/{LEAGUE_ID}"
    headers = {"X-API-KEY": API_KEY}

    r = requests.get(url, headers=headers)
    data = r.json()
    games = data.get("schedule", [])

    conn = sqlite3.connect(DB)
    cur = conn.cursor()

    for g in games:
        cur.execute("""
            INSERT OR IGNORE INTO partidos (
                external_id,
                semana,
                fecha,
                equipo_local,
                equipo_visitante,
                estadio,
                status
            ) VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (
            g.get("idEvent"),
            g.get("intRound"),          # semana (si viene)
            g.get("dateEvent"),
            g.get("strHomeTeam"),
            g.get("strAwayTeam"),
            g.get("strVenue"),
            g.get("strStatus")
        ))

    conn.commit()
    conn.close()
    print(f"{len(games)} partidos guardados.")


In [ ]:
save_next_games()

20 partidos guardados.


In [ ]:
import requests

API_KEY = "609380"  # tu clave
EVENT_ID = "1234567"  # reemplaza con un id real de partido

url = f"https://www.thesportsdb.com/api/v2/json/{API_KEY}/lookupeventresults.php?id={EVENT_ID}"
r = requests.get(url)

print(r.status_code)
print(r.text)  # ver qué devuelve realmente


In [6]:
import sqlite3
from db import DB

conn = sqlite3.connect(DB)
cur = conn.cursor()
cur.execute("SELECT partido_id, external_id FROM partidos LIMIT 5")
print(cur.fetchall())
conn.close()


[(2261410, '2261410'), (2261411, '2261411'), (2261412, '2261412'), (2261413, '2261413'), (2261414, '2261414')]


In [8]:
import requests

API_KEY = "609380"
event_id = 2261410  # ejemplo

url = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}/eventresults.php?id={event_id}"
r = requests.get(url)

print(r.status_code)
print(r.json())


200
{'results': None}
